# Creating Lists of Words Related to a Specified Search Term

## This algorithm provides the following:
- Imports a dataframe of tweets
- Processes tweets and tokenizes words
- Uses Word Embeddings to convert words into vectors
- Determines related words by using cosine similarity within the vector space
- Generates a list, or lists, of words within the time delta(s) selected, ordered by cosine similarity.

### Imported Libraries
Libraries and modules below are used to import and process the tokens into vectors, removing extraneous words.

In [ ]:
import pandas as pd
import numpy as np
from gensim.models import Word2Vec
from nltk.tokenize import TweetTokenizer
from nltk.corpus import stopwords
from math import ceil
from sklearn.metrics import f1_score, classification_report, confusion_matrix, accuracy_score, precision_score, recall_score
import string
from math import log10
from scipy.stats import norm
import re
import pickle
import matplotlib.pyplot as plt
%matplotlib inline

### Tweet Import
Importing and parsing tweets into dataframe, converting the datestamp strings to datetime objects.

In [ ]:
columns = ['tweet_id','timestamp','tweet_text','user_id',
           'tweet_coords','tweet_coords_list','tweet_long','tweet_lat','location',
           'enc_url','tweet_lang','hashtags']
tweet_full = pd.read_csv(r'./tweetCoords.csv',
                         header=None,
                         names=columns,
                         parse_dates=[1],
                         infer_datetime_format=True,
                         index_col='timestamp')

### Isolating Tweets by language (english)


In [ ]:
tweet_full_en = tweet_full[tweet_full['tweet_lang'] == 'en']

### Text Cleaning and Tokenization Functions

In [ ]:
tweet_stops = stopwords.words('english')
tweet_tokenizer = TweetTokenizer(strip_handles=True,preserve_case=False,reduce_len=True)

def clean_tweet(tweet):
#     takes input string and converts or removes characters depending on settings.
#     returns a string
#     convert case:
    tweet = tweet.lower()
#     remove URLs:
    tweet = re.sub('https?://\S+','',tweet)
#     remove @mentions, including those with a leading '-' or '.' : 
    tweet = re.sub('[-\.]?@\w+','',tweet)
#     remove punctuation, but not hashtags:
    tweet = tweet.translate(tweet.maketrans('','',string.punctuation.replace("#","")))
#     remove non-hashtag '#'.
    tweet = re.sub('#\B','',tweet)
#     remove 'amp', 'gt', 'lt', indicating decoded ampersand, greater-than, less-than characters
    tweet = re.sub(r'\b(amp|gt|lt)\b','',tweet)
#     drop numbers and words of < 4 characters.
    tweet = re.sub(r'\b\w{1,3}\b','',tweet)
    tweet = re.sub(r'\b\d+\b','',tweet)
    return tweet

def tokens_no_stopwords(tweet_as_string):
#     wrapper function that combines the tokenizer, cleaner, and stopword removal.
#     takes a string and returns a list of strings
    cleaned_tweet = clean_tweet(tweet_as_string)
    tweet_as_tokens = tweet_tokenizer.tokenize(cleaned_tweet)
    tweet_no_stops = [word for word in tweet_as_tokens if word not in tweet_stops]
    
    return tweet_no_stops

### Search Term
This is the term that will serve as the comparison for all later lists.

In [ ]:
search_term = "irma"

### Related Words Dataframe and Time Deltas
This instantiates the dataframe for the related words and specifies the start, end, and time delta for the periods of related words.

In [ ]:
related_words = pd.DataFrame()
tweet_date = pd.to_datetime("2017-09-10 00:00:00")
date_delta = pd.Timedelta("24HR")
end_date = pd.to_datetime("2017-09-10 00:00:00")

### Related Word list quantity
This number specifies the number of words that will be returned in each list associated with the time periods specified above

In [ ]:
top_num_words = 300

## Word Embeddings
### Converting Words to Vectors using the 'Word2Vec' library

- Iterate through each time period designated above
- Apply the tokenization and cleaning functions
- Convert the tokens to vectors

In [ ]:
for i in range(1,11):
    for tweet_day in pd.date_range(start = tweet_date, end = end_date, freq = date_delta):
        tweet_text = tweet_full_en.loc[tweet_day:tweet_day + date_delta,"tweet_text"]

        tweets_tokens = tweet_text.apply(tokens_no_stopwords)
#         this line uses the for loop to iterate the minimum word count for inclusion in the vectors:
        vector_model = Word2Vec(tweets_tokens, min_count=i, window=7, workers=1, size=100, seed=1, sg=1)
#         this line uses the for loop to iterate the context window size for any given word
#         vector_model = Word2Vec(tweets_tokens, min_count=1, window=i, workers=1, size=100, seed=1, sg=1)
        
        word_matrix = vector_model.wv[vector_model.wv.vocab]
        vector_model.train(tweets_tokens, total_examples=len(tweet_text), epochs=10)
        terms_from_range = pd.DataFrame.from_records(vector_model.wv.most_similar(search_term,topn=top_num_words),
                                                     columns=[tweet_day,"score"])

        related_words = pd.concat([related_words,terms_from_range],axis=1)

## Identifying Relationships with Human Coded Data

### Using the list of related words to identify an Irma Related tweet.

Importing a list of human coded tweets for 'irma related'

In [ ]:
coded_tweets = pd.read_csv(r'./irma_related_tweets.csv')

tweets_on_date = tweet_full_en.loc[tweet_date:tweet_date+date_delta]

tweet_encoded = pd.concat([coded_tweets.reset_index(),tweets_on_date.iloc[:-1].reset_index()],axis=1)

The counts of human coded Irma related tweets: 0 = not related, 1 = 'irma' related

In [ ]:
hc_encoded_counts = tweet_encoded.loc[:,['tweet_id','timestamp','irma_rel','tweet_text']].groupby('irma_rel').size()
print(hc_encoded_counts)

In [ ]:
tweets_on_date = tweet_full_en.loc[tweet_date:tweet_date+date_delta]
tweet_text = tweets_on_date.tweet_text
tweets_tokens = tweet_text.apply(tokens_no_stopwords)
#         this line uses the for loop to iterate the minimum word count for inclusion in the vectors:
vector_model = Word2Vec(tweets_tokens, min_count=1, window=6, workers=1, size=100, seed=1, sg=1)
#         this line uses the for loop to iterate the context window size for any given word
#         vector_model = Word2Vec(tweets_tokens, min_count=1, window=i, workers=1, size=100, seed=1, sg=1)
        
word_matrix = vector_model.wv[vector_model.wv.vocab]
vector_model.train(tweets_tokens, total_examples=len(tweet_text), epochs=10)

In [ ]:
def ScoreTweet(tweet):
    tweet_as_terms = tokens_no_stopwords(tweet)
    score = 0
    for i in tweet_as_terms:
        if i in vector_model.wv.vocab:
            score += vector_model.wv.similarity(i,search_term)
    if len(tweet_as_terms) > 0:
        score /= len(tweet_as_terms)
    else:
        score = 0
    return score


In [ ]:
tweet_encoded['Score'] = tweet_encoded.tweet_text.apply(ScoreTweet)

In [ ]:
tweet_encoded['MMScore'] = (tweet_encoded['Score'] - tweet_encoded['Score'].min())* 100 / (tweet_encoded['Score'].max() - tweet_encoded['Score'].min())

In [ ]:
tweet_encoded['MMScore']

In [ ]:
new_func_df = pd.DataFrame(np.zeros((101,1)))

In [ ]:
for i in range(101):
    new_func_df.iloc[i] = f1_score(tweet_encoded.irma_rel,tweet_encoded['MMScore'] > i)

In [ ]:
plt.figure(figsize=(16,8))
plt.plot(new_func_df);

In [ ]:
plt.figure(figsize=(16,8))
plt.hist(tweet_encoded['MMScore'],bins=100);

In [ ]:
[len(tweet_encoded[tweet_encoded['MMScore'] > i]) for i in range(101)]

In [ ]:
plt.figure(figsize=(16,8))
plt.plot(range(101),[len(tweet_encoded[tweet_encoded['MMScore'] > i])/19088 for i in range(101)]);

In [ ]:
pd.Series([len(tweet_encoded[tweet_encoded['MMScore'] > i])/19088 for i in range(101)]).to_csv(r'./new_func.csv')

### Calculating F1 scores

Creating a function to evaluate F1 score for each iteration of the word list.

In [ ]:
related_test = pickle.load(open('./pkl_sets/related_words_window_10E.pkl',"rb"))

In [ ]:
related_words_lists = related_test.iloc[:,0::2]

In [ ]:
def WordsCWScoreList(position,column=0):
    related_word_list = list(related_words_lists.iloc[0:position,column])
    related_word_list.append(search_term)
    return "|".join(related_word_list)

This function takes a numeric position as an input. From there it:
- finds all of the words in the cosine-similarity ordered list for the range are at that position or above.
- creates a list of those words.
- appends the search term to that list.
- converts that list to a pipe-delimited string.
If the `related_word_list` dataframe has more than one set of related words (as columns) the the second argument allows subsequent columns' lists to be returned.

This pipe-delimited string can be used as the input of the `pd.Series.str.contains()` method as a regular expression for matching rows who contain one or more of the terms.

In [ ]:
WordsCWScoreList(10,7)

For purposes of evaluating the effectiveneness of the related words threshold, there are 4 categories of tweets:
- human-coded 'irma-related' and the word list is a match (true positive)
- human-coded 'not irma-related' and the word list is a match (false positive)
- human-coded 'irma-related' and the word list is not a match (false negative)
- human-coded 'not irma-related' and the word list is not a match (true negative)

The `score_matrix` dataframe is initialized to be zeroes, to prevent contamination from previous iterations. The following for loop iterates over the `related_words_lists` dataframe from above to evaluate a tweet as related based upon containing one or more words of the subset of the word list for that particular window or minimum count.

In [ ]:
score_matrix = pd.DataFrame(np.zeros(related_words_lists.shape))

In [ ]:
f1_score(tweet_encoded.irma_rel,tweet_encoded.tweet_text.str.contains(WordsCWScoreList(239,5),flags=re.IGNORECASE))

In [ ]:
precision_score(tweet_encoded.irma_rel,tweet_encoded.tweet_text.str.contains(WordsCWScoreList(239,5),flags=re.IGNORECASE))

In [ ]:
recall_score(tweet_encoded.irma_rel,tweet_encoded.tweet_text.str.contains(WordsCWScoreList(239,5),flags=re.IGNORECASE))

In [ ]:
confusion_matrix(tweet_encoded.irma_rel,tweet_encoded.tweet_text.str.contains(WordsCWScoreList(239,5),flags=re.IGNORECASE))

In [ ]:
# The commented lines should be uncommented for debug purposes.
for i in range(0,related_words_lists.shape[1]):
#     print(pd.Timestamp.now().time())
    for j in range(0,related_words_lists.shape[0]):
        score_matrix.iloc[j,i]=f1_score(tweet_encoded.irma_rel,tweet_encoded.tweet_text.str.contains(WordsCWScoreList(j,i),flags=re.IGNORECASE))   
#     print(score_matrix.iloc[j,:])   

In [ ]:
# pickle.dump(score_matrix,open('./score_matrix_seed_term_min_count_10E.pkl',"wb"))

In [ ]:
# pickle.dump(related_words,open('./related_words_min_count_10E.pkl',"wb"))

### Analysis on Cosine Similarity Values
#### Checking lists of words and F1 scores for effectiveness of analysis based upon similar words contained.

In [ ]:
word_graph = related_test.iloc[0:239,10:12]

In [ ]:
word_graph.to_csv(r'word_values.csv')

In [ ]:
plt.plot(word_graph.iloc[:,1]);

In [ ]:
word_graph.iloc[:,1] = word_graph.iloc[:,1].apply(log10)

In [ ]:
cos_sim_values = word_graph.iloc[:,1]

In [ ]:
word_graph.iloc[:,1] = (cos_sim_values - cos_sim_values.min())* 100 / (cos_sim_values.max() - cos_sim_values.min())

In [ ]:
word_graph.head(10)

In [ ]:
word_graph.head(10)

In [ ]:
def WordsMMScoreList(score,scored_list):
    word_list = list(scored_list[scored_list.iloc[:,1] >= score].iloc[:,0])
    word_list.append(search_term)
    return "|".join(word_list)

In [ ]:
WordsMMScoreList(100,word_graph)

In [ ]:
# score_matrix = pickle.load(open('./pkl_sets/score_matrix_seed_term_window_10E.pkl',"rb"))

In [ ]:
len(tweet_encoded[tweet_encoded.tweet_text.str.contains('pass')])

In [ ]:
min_max_chart = pd.DataFrame(np.zeros((101,1)))

In [ ]:
for i in range(101):
    min_max_chart.iloc[i] = len(tweet_encoded[tweet_encoded.tweet_text.str.contains(WordsMMScoreList(i,word_graph))])

In [ ]:
min_max_chart.to_csv(r'./min_max_chart.csv')

In [ ]:
min_max_chart = min_max_chart/8048

In [ ]:
fig = plt.figure(figsize=(16,8))
plt.plot(min_max_chart);

In [ ]:
score_matrix.iloc[235:240,5]

In [ ]:
# np.all(score_matrix == score_matrix_test)

In [ ]:
# for i in range(0,top_num_words,10):
#     print(str(i)+": ",end="")
#     print(f1_score(tweet_encoded.irma_rel,tweet_encoded.tweet_text.str.contains(WordsCWScoreList(i),flags=re.IGNORECASE)))      

In [ ]:
# accuracy_score(tweet_encoded.irma_rel,tweet_encoded.tweet_text.str.contains(WordsCWScoreList(1),flags=re.IGNORECASE))

In [ ]:
import matplotlib.pyplot as plt
%matplotlib inline

In [ ]:
fig = plt.figure(figsize=(16,8))
plt.title
plt.plot()

In [ ]:
fig = plt.figure(figsize=(16,8))
plt.title("Word Vector Analysis of Terms By Window Size Trained in 10 Epochs")
plt.xlabel("Number of Terms (Including Search Term)")
plt.ylabel("F1 Score: Identifying Relatedness by Term Inclusion")
# for i in (range(1,11)):
#     plt.plot(score_matrix.iloc[1:,i-1],label="min count: "+str(i))
i=6
plt.plot(score_matrix.iloc[1:,i-1],label="min count: "+str(i))
plt.axvline(238)
plt.legend();

In [ ]:
conf_df = pd.DataFrame(np.zeros((300,4)))

In [ ]:
for i in range(300):
    conf_df.iloc[i] = pd.Series(confusion_matrix(tweet_encoded.irma_rel,tweet_encoded.tweet_text.str.contains(WordsCWScoreList(i,5),flags=re.IGNORECASE)).ravel())
    

In [ ]:
fig = plt.figure(figsize=(16,8))
plt.plot(conf_df.iloc[1:,3],label="True Positive") 
plt.plot(conf_df.iloc[1:,2],label="False Negative") 
plt.plot(conf_df.iloc[1:,1],label="False Positive")
plt.plot(conf_df.iloc[1:,0],label="True Negative")
plt.legend();

In [ ]:
#3 = true positive
#0 = true negative
#1 = false positive
#2 = false negative

In [ ]:
fig = plt.figure(figsize=(16,8))
ax = fig.subplots()

color_list = ["#562a72", "#862d63", "#aaa939", "#7e9f35"]

ax.stackplot(conf_df.iloc[1:,:].index, 
             conf_df.iloc[1:,3], 
             conf_df.iloc[1:,2], 
             conf_df.iloc[1:,1], 
             conf_df.iloc[1:,0], 
             labels=["True Positive","False Negative","False Positive","True Negative"],
            colors = color_list)
# ax.axhline(hc_encoded_counts[1],linestyle="-.",color="w")
ax.axvline(238,linestyle="-.",color="k",label="Optimal F1 Score")
ax.legend(loc='upper left')
plt.show()

In the chart below, an increase in x represents a longer list of words whose inclusion would indicate an 'irma-related' tweet within the model. By allowing more words into this list, there is a larger number of tweets evaluated as a true positive result, but it also increases the false positives. The objective is to maximize the number of true positives, while minimizing false positives.

In [ ]:
# fig = plt.figure(figsize=(16,8))
# ax = fig.subplots()

# color_list = ["#562a72", "#862d63", "#aaa939", "#7e9f35"]

# ax.stackplot(score_matrix.iloc[1:,:].index, 
#              score_matrix.iloc[1:,:].TruePos, 
#              score_matrix.iloc[1:,:].FalseNeg, 
#              score_matrix.iloc[1:,:].FalsePos, 
#              score_matrix.iloc[1:,:].TrueNeg,
#              labels=["True Positive","False Negative","False Positive","True Negative"],
#             colors = color_list)
# ax.axhline(hc_encoded_counts[1],linestyle="-.",color="w")
# ax.legend(loc='upper left')
# plt.show()

In [ ]:
# print(score_matrix.iloc[ceil(related_words.iloc[50].score)])